In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf
import scipy.stats as stats
import pandas_profiling   

from sklearn.model_selection import train_test_split
from sklearn import metrics
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
bankloans = pd.read_csv(r'C:\Users\Chinthana\Desktop\Analytix Labs\practice\Python\class 19\Python (BLR) BTCMP AUG 2022 _ Class 19 Python (BLR) BTCMP AUG 2022\3. Logistic Regression\1. Case Study - Bank Loans - Class Exercise\bankloans.csv' )

In [3]:
bankloans

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default
0,41,3,17,12,176,9.3,11.359392,5.008608,1.0
1,27,1,10,6,31,17.3,1.362202,4.000798,0.0
2,40,1,15,14,55,5.5,0.856075,2.168925,0.0
3,41,1,15,14,120,2.9,2.658720,0.821280,0.0
4,24,2,2,0,28,17.3,1.787436,3.056564,1.0
...,...,...,...,...,...,...,...,...,...
845,34,1,12,15,32,2.7,0.239328,0.624672,NaN
846,32,2,12,11,116,5.7,4.026708,2.585292,NaN
847,48,1,13,11,38,10.8,0.722304,3.381696,NaN
848,35,2,1,11,24,7.8,0.417456,1.454544,NaN


In [4]:
pwd

'C:\\Users\\Chinthana\\Desktop\\Analytix Labs\\practice\\Python\\class 19\\mine'

In [5]:
profile_report = pandas_profiling.ProfileReport(bankloans)
profile_report.to_file('profileReport.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
bankloans_existing = bankloans[bankloans.default.isnull()==0]
bankloans_new = bankloans[bankloans.default.isnull()==1]

In [7]:
bankloans_existing

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default
0,41,3,17,12,176,9.3,11.359392,5.008608,1.0
1,27,1,10,6,31,17.3,1.362202,4.000798,0.0
2,40,1,15,14,55,5.5,0.856075,2.168925,0.0
3,41,1,15,14,120,2.9,2.658720,0.821280,0.0
4,24,2,2,0,28,17.3,1.787436,3.056564,1.0
...,...,...,...,...,...,...,...,...,...
695,36,2,6,15,27,4.6,0.262062,0.979938,1.0
696,29,2,6,4,21,11.5,0.369495,2.045505,0.0
697,33,1,15,3,32,7.6,0.491264,1.940736,0.0
698,45,1,19,22,77,8.4,2.302608,4.165392,0.0


In [8]:
bankloans_new

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default
700,36,1,16,13,32,10.9,0.544128,2.943872,NaN
701,50,1,6,27,21,12.9,1.316574,1.392426,NaN
702,40,1,9,9,33,17.0,4.880700,0.729300,NaN
703,31,1,5,7,23,2.0,0.046000,0.414000,NaN
704,29,1,4,0,24,7.8,0.866736,1.005264,NaN
...,...,...,...,...,...,...,...,...,...
845,34,1,12,15,32,2.7,0.239328,0.624672,NaN
846,32,2,12,11,116,5.7,4.026708,2.585292,NaN
847,48,1,13,11,38,10.8,0.722304,3.381696,NaN
848,35,2,1,11,24,7.8,0.417456,1.454544,NaN


In [9]:
bankloans_existing.age.mode()

0    29
Name: age, dtype: int64

In [10]:
def out(x):
    x.clip(lower=x.quantile(0.01), upper=x.quantile(0.99))
    return x

In [11]:
def missings_treat(x):
    x = x.fillna(x.median())
    return x



In [12]:
bankloans_existing=bankloans_existing.apply(missings_treat)
bankloans_existing=bankloans_existing.apply(out)

In [13]:
bankloans_existing

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default
0,41,3,17,12,176,9.3,11.359392,5.008608,1.0
1,27,1,10,6,31,17.3,1.362202,4.000798,0.0
2,40,1,15,14,55,5.5,0.856075,2.168925,0.0
3,41,1,15,14,120,2.9,2.658720,0.821280,0.0
4,24,2,2,0,28,17.3,1.787436,3.056564,1.0
...,...,...,...,...,...,...,...,...,...
695,36,2,6,15,27,4.6,0.262062,0.979938,1.0
696,29,2,6,4,21,11.5,0.369495,2.045505,0.0
697,33,1,15,3,32,7.6,0.491264,1.940736,0.0
698,45,1,19,22,77,8.4,2.302608,4.165392,0.0


In [14]:
#Information value calculation
def calculate_woe_iv(dataset, feature, target):
    lst = []
    for i in range(dataset[feature].nunique()):
        val = list(dataset[feature].unique())[i]
        lst.append({
            'Value': val,
            'All': dataset[dataset[feature] == val].count()[feature],
            'Good': dataset[(dataset[feature] == val) & (dataset[target] == 0)].count()[feature],
            'Bad': dataset[(dataset[feature] == val) & (dataset[target] == 1)].count()[feature]
        })
        
    dset = pd.DataFrame(lst)
    dset['Distr_Good'] = dset['Good'] / dset['Good'].sum()
    dset['Distr_Bad'] = dset['Bad'] / dset['Bad'].sum()
    dset['WoE'] = np.log(dset['Distr_Good'] / dset['Distr_Bad'])
    dset = dset.replace({'WoE': {np.inf: 0, -np.inf: 0}})
    dset['IV'] = (dset['Distr_Good'] - dset['Distr_Bad']) * dset['WoE']
    iv = dset['IV'].sum()
    
    dset = dset.sort_values(by='WoE')
    
    return dset, iv

In [15]:
for col in bankloans_existing.columns:
    if col == 'default': continue
    else:
        print('WoE and IV for column: {}'.format(col))
        df, iv = calculate_woe_iv(bankloans_existing, col, 'default')
        #print(df)
        print('IV score: {:.2f}'.format(iv))
        print('\n')

WoE and IV for column: age
IV score: 0.42


WoE and IV for column: ed
IV score: 0.08


WoE and IV for column: employ


C:\Users\Chinthana\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Chinthana\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


IV score: 0.60


WoE and IV for column: address
IV score: 0.45


WoE and IV for column: income


C:\Users\Chinthana\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


IV score: 0.40


WoE and IV for column: debtinc


C:\Users\Chinthana\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


IV score: 0.49


WoE and IV for column: creddebt


C:\Users\Chinthana\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


IV score: 0.00


WoE and IV for column: othdebt
IV score: 0.00




C:\Users\Chinthana\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [16]:
#important variables using informaiton value: age, employ, address, income, debtinc

In [17]:
model = smf.logit(formula = 'default~employ', data=bankloans_existing).fit()

Optimization terminated successfully.
         Current function value: 0.528154
         Iterations 6


In [18]:
print(model.summary())

                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                  700
Model:                          Logit   Df Residuals:                      698
Method:                           MLE   Df Model:                            1
Date:                Thu, 30 Mar 2023   Pseudo R-squ.:                 0.08075
Time:                        11:53:01   Log-Likelihood:                -369.71
converged:                       True   LL-Null:                       -402.18
Covariance Type:            nonrobust   LLR p-value:                 7.685e-16
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1502      0.138     -1.087      0.277      -0.421       0.121
employ        -0.1247      0.017     -7.149      0.000      -0.159      -0.091


In [19]:
pred_proba=model.predict(bankloans_existing)

In [89]:
#SomerceD
2*metrics.roc_auc_score(bankloans_existing.default, pred_proba)-1

0.40913847227066613

In [90]:
gini_df = pd.DataFrame()
for col in bankloans_existing.columns.difference(['default']):
    model = smf.logit('default~'+str(col), data=bankloans_existing).fit()
    gini = 2*metrics.roc_auc_score(bankloans_existing.default, model.predict(bankloans_existing))-1
    temp = pd.DataFrame([col, gini]).T
    gini_df = pd.concat([gini_df, temp], axis=0)

Optimization terminated successfully.
         Current function value: 0.559938
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.564806
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.547045
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.501021
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.568222
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.528154
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.571691
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.564828
         Iterations 5


In [91]:
gini_df.columns=['var', 'gini']

In [92]:
gini_df.sort_values(by=['gini'], ascending=False)

,var,gini
0,debtinc,0.470601
0,employ,0.409138
0,creddebt,0.272378
0,address,0.222384
0,age,0.206403
0,income,0.200336
0,othdebt,0.173828
0,ed,0.147964


In [30]:
#cutoff  gini>0.20

In [93]:
bank_1=bankloans_existing[[ 'employ', 'address', 'debtinc', 'creddebt','age', 'income']]
#drop age and income as they have high vif value

In [94]:
variance_inflation_factor(bank_1.values, 0)

4.754634249329137

In [95]:
vif=pd.DataFrame()
vif['vif_val']=[variance_inflation_factor(bank_1.values, i) for i in range(bank_1.shape[1])]
vif['col']=bank_1.columns

In [96]:
vif.sort_values(by='vif_val', ascending=False)

,vif_val,col
4,10.936917,age
5,6.088224,income
2,4.855113,debtinc
0,4.754634,employ
3,3.666716,creddebt
1,3.465390,address


In [97]:
#drop age and income as they have high vif value

In [98]:
bank_final = bankloans_existing[['creddebt', 'employ', 'debtinc', 'address', 'default']]

In [99]:
#split the data into train & test
train, test=train_test_split(bank_final, test_size=0.30, random_state=13422)

In [100]:
train

,creddebt,employ,debtinc,address,default
192,0.815360,10,11.2,0,1.0
171,0.291088,4,11.3,10,0.0
411,1.256346,22,11.7,24,0.0
556,0.125334,7,2.2,0,0.0
493,5.715360,12,16.0,14,1.0
...,...,...,...,...,...
309,1.060500,4,10.1,19,0.0
620,0.190095,1,6.9,1,0.0
198,0.398889,8,6.9,27,0.0
452,2.560779,11,12.9,17,0.0


In [101]:
test

,creddebt,employ,debtinc,address,default
128,0.366750,5,9.0,1,0.0
68,1.492154,8,8.2,3,1.0
619,4.637556,7,15.4,17,0.0
657,1.143135,16,10.5,10,0.0
590,4.960032,10,13.2,1,1.0
...,...,...,...,...,...
78,0.199134,14,2.6,2,0.0
117,1.032512,15,3.4,16,0.0
576,0.569296,8,5.6,0,1.0
15,0.817516,9,8.6,6,1.0


In [102]:
'default~'+'+'.join(bank_final.columns.difference(['default']))

'default~address+creddebt+debtinc+employ'

In [103]:
#model building
logit_model=smf.logit(data=train, formula='default~'+'+'.join(train.columns.difference(['default']))).fit()

Optimization terminated successfully.
         Current function value: 0.374257
         Iterations 7


In [104]:
print(logit_model.summary())

                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                  490
Model:                          Logit   Df Residuals:                      485
Method:                           MLE   Df Model:                            4
Date:                Thu, 30 Mar 2023   Pseudo R-squ.:                  0.3249
Time:                        13:02:25   Log-Likelihood:                -183.39
converged:                       True   LL-Null:                       -271.63
Covariance Type:            nonrobust   LLR p-value:                 4.225e-37
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.8618      0.322     -2.676      0.007      -1.493      -0.231
address       -0.0914      0.025     -3.627      0.000      -0.141      -0.042
creddebt       0.6219      0.111      5.590      0.0

In [105]:
#Mathematical Equation
# LE = -0.2479*employ + 0.6219 *creddebt + 0.0891*debtinc-0.0914*address-0.868
# P(default=1) = exp(LE)/(1+exp(LE))

#Positive drivers: creddebt, debtinc
#Negative drivers: address, employ

In [106]:
train['prde_prob']=logit_model.predict(train)
test['pred_prob']=logit_model.predict(test)

In [107]:
print("train_roc:",metrics.roc_auc_score(train.default,train['prde_prob'] ))
print("test_roc:",metrics.roc_auc_score(test.default,test['pred_prob'] ))
print("train_gini:",2*metrics.roc_auc_score(train.default,train['prde_prob'] )-1)
print("test_gini:",2*metrics.roc_auc_score(test.default,test['pred_prob'] )-1)

train_roc: 0.8637341729144487
test_roc: 0.8314426369863014
train_gini: 0.7274683458288973
test_gini: 0.6628852739726028


In [108]:
#cut-off
train.default.mean()  #percentage of 1's

0.24285714285714285

In [109]:
cf=pd.DataFrame()
for cutoff in np.linspace(0,1,50):
    train['pred']=np.where(train.prde_prob>cutoff, 1, 0)
    
    TP=np.where((train.default==1) & (train.pred==1), 1, 0)
    TN=np.where((train.default==0) & (train.pred==0), 1, 0)
    FP=np.where((train.default==0) & (train.pred==1), 1, 0)
    FN=np.where((train.default==1) & (train.pred==0), 1, 0)
    
    actual_1s=(TP.sum()+FN.sum())
    actual_0s=(TN.sum()+FP.sum())
    
    sensitivity=TP.sum()/actual_1s
    specificity=TN.sum()/(TN.sum()+FP.sum())
    Total=sensitivity+specificity
    
    temp=pd.DataFrame([cutoff, specificity, sensitivity, Total]).T
    cf=pd.concat([cf,temp], axis=0)
    

In [110]:
cf.columns = ['cutoff', 'Specicity', 'sensitivity', 'Total']

In [112]:
cf

,cutoff,Specicity,sensitivity,Total
0,0.000000,0.000000,1.000000,1.000000
0,0.020408,0.223720,0.991597,1.215316
0,0.040816,0.320755,0.983193,1.303948
0,0.061224,0.398922,0.957983,1.356905
0,0.081633,0.455526,0.932773,1.388299
0,0.102041,0.520216,0.924370,1.444585
0,0.122449,0.555256,0.915966,1.471222
0,0.142857,0.611860,0.899160,1.511020
0,0.163265,0.652291,0.899160,1.551451
0,0.183673,0.692722,0.848739,1.541462


In [113]:
cf[cf.Total==cf.Total.max()]

,cutoff,Specicity,sensitivity,Total
0,0.265306,0.803235,0.781513,1.584747


In [124]:
optimal_cutoff=cf.sort_values(by='Total', ascending=False).head(1).cutoff.values[0]

In [125]:
train

,creddebt,employ,debtinc,address,default,prde_prob,pred
192,0.815360,10,11.2,0,1.0,0.137492,0
171,0.291088,4,11.3,10,0.0,0.170829,0
411,1.256346,22,11.7,24,0.0,0.001247,0
556,0.125334,7,2.2,0,0.0,0.089206,0
493,5.715360,12,16.0,14,1.0,0.465910,0
...,...,...,...,...,...,...,...
309,1.060500,4,10.1,19,0.0,0.116023,0
620,0.190095,1,6.9,1,0.0,0.385027,0
198,0.398889,8,6.9,27,0.0,0.011544,0
452,2.560779,11,12.9,17,0.0,0.083108,0


In [126]:
train['pred']=np.where(train.prde_prob>optimal_cutoff,1,0)
test['pred']=np.where(test.pred_prob>optimal_cutoff,1,0)

In [127]:
train

,creddebt,employ,debtinc,address,default,prde_prob,pred
192,0.815360,10,11.2,0,1.0,0.137492,0
171,0.291088,4,11.3,10,0.0,0.170829,0
411,1.256346,22,11.7,24,0.0,0.001247,0
556,0.125334,7,2.2,0,0.0,0.089206,0
493,5.715360,12,16.0,14,1.0,0.465910,1
...,...,...,...,...,...,...,...
309,1.060500,4,10.1,19,0.0,0.116023,0
620,0.190095,1,6.9,1,0.0,0.385027,1
198,0.398889,8,6.9,27,0.0,0.011544,0
452,2.560779,11,12.9,17,0.0,0.083108,0


In [128]:
test

,creddebt,employ,debtinc,address,default,pred_prob,pred
128,0.366750,5,9.0,1,0.0,0.238118,0
68,1.492154,8,8.2,3,1.0,0.188326,0
619,4.637556,7,15.4,17,0.0,0.526257,1
657,1.143135,16,10.5,10,0.0,0.016364,0
590,4.960032,10,13.2,1,1.0,0.695962,1
...,...,...,...,...,...,...,...
78,0.199134,14,2.6,2,0.0,0.015367,0
117,1.032512,15,3.4,16,0.0,0.006073,0
576,0.569296,8,5.6,0,1.0,0.120013,0
15,0.817516,9,8.6,6,1.0,0.085728,0


In [129]:
metrics.confusion_matrix(train.default, train.pred)

array([[298,  73],
       [ 26,  93]], dtype=int64)

In [130]:
metrics.confusion_matrix(test.default, test.pred)

array([[106,  40],
       [ 14,  50]], dtype=int64)

In [131]:
print(metrics.classification_report(train.default, train.pred))

              precision    recall  f1-score   support

         0.0       0.92      0.80      0.86       371
         1.0       0.56      0.78      0.65       119

    accuracy                           0.80       490
   macro avg       0.74      0.79      0.76       490
weighted avg       0.83      0.80      0.81       490



In [132]:
print(metrics.classification_report(test.default, test.pred))

              precision    recall  f1-score   support

         0.0       0.88      0.73      0.80       146
         1.0       0.56      0.78      0.65        64

    accuracy                           0.74       210
   macro avg       0.72      0.75      0.72       210
weighted avg       0.78      0.74      0.75       210



In [133]:
train['cred_score'] = (1-train.prde_prob)*1000
test['cred_score'] = (1-test.pred_prob)*1000

In [134]:
train

,creddebt,employ,debtinc,address,default,prde_prob,pred,cred_score
192,0.815360,10,11.2,0,1.0,0.137492,0,862.507603
171,0.291088,4,11.3,10,0.0,0.170829,0,829.171423
411,1.256346,22,11.7,24,0.0,0.001247,0,998.753121
556,0.125334,7,2.2,0,0.0,0.089206,0,910.793885
493,5.715360,12,16.0,14,1.0,0.465910,1,534.089860
...,...,...,...,...,...,...,...,...
309,1.060500,4,10.1,19,0.0,0.116023,0,883.977130
620,0.190095,1,6.9,1,0.0,0.385027,1,614.973493
198,0.398889,8,6.9,27,0.0,0.011544,0,988.455826
452,2.560779,11,12.9,17,0.0,0.083108,0,916.892235


In [135]:
bankloans_new['pred_prob']=logit_model.predict(bankloans_new)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_16084\2314354713.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bankloans_new['pred_prob']=logit_model.predict(bankloans_new)


In [136]:
bankloans_new

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default,pred_prob
700,36,1,16,13,32,10.9,0.544128,2.943872,NaN,0.008949
701,50,1,6,27,21,12.9,1.316574,1.392426,NaN,0.054738
702,40,1,9,9,33,17.0,4.880700,0.729300,NaN,0.653425
703,31,1,5,7,23,2.0,0.046000,0.414000,NaN,0.073482
704,29,1,4,0,24,7.8,0.866736,1.005264,NaN,0.349859
...,...,...,...,...,...,...,...,...,...,...
845,34,1,12,15,32,2.7,0.239328,0.624672,NaN,0.008015
846,32,2,12,11,116,5.7,4.026708,2.585292,NaN,0.138222
847,48,1,13,11,38,10.8,0.722304,3.381696,NaN,0.024631
848,35,2,1,11,24,7.8,0.417456,1.454544,NaN,0.238563


In [137]:
bankloans_new['credit_score']=(1-bankloans_new['pred_prob'])*1000

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_16084\2902022240.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bankloans_new['credit_score']=(1-bankloans_new['pred_prob'])*1000


In [138]:
bankloans_new

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default,pred_prob,credit_score
700,36,1,16,13,32,10.9,0.544128,2.943872,NaN,0.008949,991.051358
701,50,1,6,27,21,12.9,1.316574,1.392426,NaN,0.054738,945.262457
702,40,1,9,9,33,17.0,4.880700,0.729300,NaN,0.653425,346.575343
703,31,1,5,7,23,2.0,0.046000,0.414000,NaN,0.073482,926.518034
704,29,1,4,0,24,7.8,0.866736,1.005264,NaN,0.349859,650.141140
...,...,...,...,...,...,...,...,...,...,...,...
845,34,1,12,15,32,2.7,0.239328,0.624672,NaN,0.008015,991.984789
846,32,2,12,11,116,5.7,4.026708,2.585292,NaN,0.138222,861.777729
847,48,1,13,11,38,10.8,0.722304,3.381696,NaN,0.024631,975.369179
848,35,2,1,11,24,7.8,0.417456,1.454544,NaN,0.238563,761.437340


In [182]:
optimal_score=(1-optimal_cutoff)*1000
optimal_score

734.6938775510205

In [177]:
bankloans_new['acceptance_flag']=np.where(bankloans_new['credit_score']>735, 'Good-accept', 'Bad-Reject')

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_12164\2424381079.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bankloans_new['acceptance_flag']=np.where(bankloans_new['credit_score']>735, 'Good-accept', 'Bad-Reject')


In [178]:
bankloans_new

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default,pred_prob,credit_score,acceptance_flag
700,36,1,16,13,32,10.9,0.544128,2.943872,NaN,0.008949,991.051358,Good-accept
701,50,1,6,27,21,12.9,1.316574,1.392426,NaN,0.054738,945.262457,Good-accept
702,40,1,9,9,33,17.0,4.880700,0.729300,NaN,0.653425,346.575343,Bad-Reject
703,31,1,5,7,23,2.0,0.046000,0.414000,NaN,0.073482,926.518034,Good-accept
704,29,1,4,0,24,7.8,0.866736,1.005264,NaN,0.349859,650.141140,Bad-Reject
...,...,...,...,...,...,...,...,...,...,...,...,...
845,34,1,12,15,32,2.7,0.239328,0.624672,NaN,0.008015,991.984789,Good-accept
846,32,2,12,11,116,5.7,4.026708,2.585292,NaN,0.138222,861.777729,Good-accept
847,48,1,13,11,38,10.8,0.722304,3.381696,NaN,0.024631,975.369179,Good-accept
848,35,2,1,11,24,7.8,0.417456,1.454544,NaN,0.238563,761.437340,Good-accept


In [179]:
bankloans_new['acceptance_flag'].value_counts()

Good-accept    100
Bad-Reject      50
Name: acceptance_flag, dtype: int64